This notebook illustrates the usage of stage-2 features as described in "Deep learning models for histologic grading of breast cancer and association with disease prognosis".

In [ ]:
import numpy as np

# Mitotic Count

In [ ]:
def generate_mitotic_heatmap(heatmap_size, list_of_coordinates, mitosis_size=6):
  """Generates mitotic heatmap with the given list of coordinates.
  
  Args:
    heatmap_size: size of the heatmap to generate
    list_of_coordinate: coordinates (tuple) of center of the mitoses.
    mitosis_size: size of each mitosis.
  Returns:
    Heatmaps that represent mitosis detection.
  """
  half_mitosis_size = int(mitosis_size / 2)
  heatmap = np.zeros(heatmap_size)
  for coord in list_of_coordinates:
    y, x = coord
    y = y - half_mitosis_size
    x = x - half_mitosis_size
    heatmap[y :(y + mitosis_size - 1), x:(x + mitosis_size - 1)] = 1
  return heatmap


def detect_and_calc_density(heatmap,
                            detection_th=0.5,
                            morph_erode_size=4,
                            window_size=128,
                            stride=64):
  """Combined steps of detection and density calculation.

  Args:
    heatmap: 2D array of shape (height, width) that represent probability of
      mitotic activity.
    detection_th: detection threshold, see mc_util.heatmap_to_list.
    morph_erode_size: size of structuring element for detection cleanup, see
      mc_util.heatmap_to_list.
    stride: density window stride, see mc_util.calculate_density.
    window_size: density window size, see mc_util.calculate_density.

  Returns:
    Dict of detection and density.
  """
  # Resize mask so it is in the same size as the heatmap.

  detection = mc_util.heatmap_to_list(
      heatmap,
      detection_th,
      morph_erode_size=morph_erode_size)
  heatmap_size = heatmap.shape

  density = mc_util.calculate_density(
      detection, heatmap_size, window_size, stride)
  return {'density': density, 'detection': detection}

In [ ]:
np.random.seed(0)

heatmap_size = (1024, 1024)
n_mitosis = 100
list_of_mitosis = [(np.random.randint(256, 768), np.random.randint(256, 768)) for _ in range(n_mitosis)]


heatmap = generate_mitotic_heatmap(heatmap_size, list_of_mitosis)
res = detect_and_calc_density(heatmap)
mc_features = stage2_features.mc_featurizer(res)
detected_mitosis = [(int(x[0]), int(x[1])) for x in res['detection']]
print('Input mitosis list:   ', sorted(list_of_mitosis, key=lambda x: x[0])[:10])
print('Detected mitosis list:',sorted(detected_mitosis, key=lambda x: x[0])[:10])
print('calculated_features:', mc_features)


Input mitosis list:    [(256, 650), (265, 467), (267, 514), (279, 443), (287, 458), (288, 438), (294, 744), (297, 314), (298, 627), (299, 616)]
Detected mitosis list: [(255, 649), (264, 466), (266, 513), (278, 442), (286, 457), (287, 437), (293, 743), (296, 313), (297, 626), (298, 615)]
calculated_features: [0.         0.         0.         0.00012207 0.0004425 ]


# Nuclear Pleomorphism and Tubule Formation

In [ ]:
ic_heatmap = [
  np.zeros((5, 4)),
[
    [np.nan, np.nan, np.nan, np.nan,],
    [1, 1, 1, 0],
    [1, 1, 1, 0],
    [1, 1, 1, 0],
    [1, np.nan, np.nan, np.nan,],
]
]
# NP/TF 1 heatmap: 0.4 of IC area, 0.33 of non-IC area
nptf1_heatmap = [
    [0, 0, 0, 0],
    [1, 1, 1, 1],
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
]
# NP/TF 2 heatmap: 0.2 of IC area, 0.66 of non-IC area
nptf2_heatmap = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 1, 1, 1],
    [0, 0, 0, 1],
    [0, 0, 0, 0],
]
# NP/TF 3 heatmap: 0.4 of IC area, 0.0 of non-IC area
nptf3_heatmap = [
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [1, 1, 1, 0],
    [1, 0, 0, 0],
]

# IC Heatmap is expected to be 3D with last channel representing the probability of being invasive carcinoma.
ic_heatmap = np.dstack(ic_heatmap)
nptf_heatmap = np.dstack([nptf1_heatmap, nptf2_heatmap, nptf3_heatmap])

tmap = {'ic_heatmap': ic_heatmap, 'heatmap': nptf_heatmap}

In [ ]:
stage2_features.np_tf_featurizer(tmap)

array([0.4       , 0.2       , 0.4       , 0.33333333, 0.66666667,
       0.        ])